In [44]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as random

In [2]:
#Define function to extract data from file, Load training data and convert into Numpy array
def get_data(file_name):
    df_input=pd.read_csv(file_name,usecols =['T_out','Press_mm_hg','Windspeed'])
    df_target=pd.read_csv(file_name,usecols =['Appliances'])
    print(df_input.info())
    print(df_target.info())
    X_train = df_input.as_matrix()
    y_train = df_target.as_matrix()
    X_train.astype('float32', copy=False)
    y_train.astype('float32', copy=False)
    return X_train,y_train


In [3]:
#Get training data
file_name = './Appliances_energy_prediction_data_set_UCI.csv'
X_train,y_train = get_data(file_name)
print('Shape of input data set',X_train.shape)
print('Shape of target data set',y_train.shape)
print('Number of samples:',len(X_train))
X_train_reshape = np.resize(X_train,(19735,2))
y_train_reshape = np.resize(y_train,(19735,1))
#print(X_train_reshape.shape)
#print(y_train_reshape.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 3 columns):
T_out          19735 non-null float64
Press_mm_hg    19735 non-null float64
Windspeed      19735 non-null float64
dtypes: float64(3)
memory usage: 462.6 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 1 columns):
Appliances    19735 non-null int64
dtypes: int64(1)
memory usage: 154.3 KB
None
Shape of input data set (19735, 3)
Shape of target data set (19735, 1)
Number of samples: 19735


In [49]:
X_train[1,:]
#y_train[1,:]

array([   6.48      ,  733.6       ,    6.66666667])

In [4]:
#Function to normalize input data along features
def normalize_data(raw_data):
    import tensorflow as tf
    return tf.nn.l2_normalize(raw_data,dim=0)

In [59]:
#Training parametres
epochs = 10
learning_rate = 0.001

In [60]:
#Network Parameteres
L1 = 10 # Number of neurons in Layer 1

In [61]:
#Placeholders for input and output
Xfeatures = 3
Yfeatures =1
#Input
X = tf.placeholder(tf.float32,shape=(None,Xfeatures))#Place holder for input data
#Output
Y = tf.placeholder(tf.float32,shape=(None,Yfeatures))   #Place holder for target output

In [63]:
#Builda single layer model using basic TensorFlow operations
def FFNN_using_core(Xinput):          #Function containing FFNN model
    #Layer1 
    W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1])) # Weights
    b_fc1 = tf.Variable(tf.random_uniform([L1]))        #Biases
    matmul_fc1=tf.matmul(Xinput, W_fc1) + b_fc1                #Matrix multiplication and additon
    h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation function

    #Output layer 
    W_fO= tf.Variable(tf.random_uniform([L1,Yfeatures])) #  # Weights
    b_fO = tf.Variable(tf.random_uniform([Yfeatures]))    #Biases
    matmul_fcO=tf.matmul(h_fc1, W_fO) + b_fO    #Matrix multiplication and additon
    output_layer = matmul_fcO  #linear activation
    return output_layer

prediction1 = FFNN_using_core(normalize_data(X))       #Call function containing for model
#prediction = FFNN_using_core(X)       #Call function
MAE1= tf.metrics.mean_absolute_error(Y,prediction1)   #Function to calculate mean absolute error 

#Loss function
loss1 = tf.reduce_mean(tf.square(Y-prediction1))

#Specify optimizer
train_step1 = tf.train.AdamOptimizer(learning_rate).minimize(loss1)

In [64]:
#Build single layer model using TensorFlow layers
def FFNN_using_layer(Xinput):          #Function containing FFNN model
    #Layer1 
    FC1 = tf.layers.dense(Xinput,L1,activation=tf.nn.relu)

    #Output layer 
    output_layer = tf.layers.dense(FC1,Yfeatures)
    return output_layer

prediction2 = FFNN_using_layer(normalize_data(X))       #Call function
#prediction = FFNN_using_layer(X)       #Call function
MAE2= tf.metrics.mean_absolute_error(Y,prediction2)
#Specify Loss function
loss2 = tf.losses.mean_squared_error (Y,prediction2)
#Specify optimizer
train_step2 = tf.train.AdamOptimizer(learning_rate).minimize(loss2)

In [65]:
#Training session
init = tf.global_variables_initializer()
init_local = tf.local_variables_initializer() 
with tf.Session() as sess:
    sess.run([init,init_local])
    for i in range(epochs):
        training_loss,_ = sess.run([loss1,train_step1],feed_dict={X:X_train,Y:y_train})    
        print("Training loss at epoch {}:{}".format(i+1,training_loss))
    _,mae = sess.run(MAE2,feed_dict={X:X_train,Y:y_train})
    print("Mean Absolute Error:",mae)

Training loss at epoch 1:19422.548828125
Training loss at epoch 2:19420.2890625
Training loss at epoch 3:19417.935546875
Training loss at epoch 4:19415.56640625
Training loss at epoch 5:19413.318359375
Training loss at epoch 6:19411.123046875
Training loss at epoch 7:19409.0078125
Training loss at epoch 8:19406.767578125
Training loss at epoch 9:19404.353515625
Training loss at epoch 10:19401.986328125
Mean Absolute Error: 97.6954


In [66]:
#Inference session
with tf.Session() as sess:
    sess.run([init,init_local])
    index = random.randint(0, len(X_train))  #Find random input vector from dataset 
    prediction = sess.run(prediction1,feed_dict={X:[X_train[index,:]],Y:[y_train[index,:]]})
    print("Predicted load for input vector at index {} {}:{}".format(index,X_train[index,:],prediction))
    _,mae = sess.run(MAE2,feed_dict={X:[X_train[index,:]],Y:[y_train[index,:]]})
    print("Actual value is {} and Mean Absolute Error is {}".format(y_train[index,:],mae))

Predicted load for input vector at index 5883 [  11.4  756.2    9.5]:[[ 10.32501793]]
Actual value is [100] and Mean Absolute Error is 100.38203430175781


In [67]:
#Building and training a single layer model using Keras (Available within TensorFlow)
model = tf.contrib.keras.models.Sequential() 
#Input Layer
model.add(tf.contrib.keras.layers.InputLayer(input_shape=(Xfeatures,),name='InputLayer'))
model.add(tf.contrib.keras.layers.BatchNormalization(axis=1))  #Normalizing values
#model.add(tf.contrib.keras.layers.Lambda(normalize_data ,name='Normalizing'))  #Normalizing values
#Layer1 
model.add(tf.contrib.keras.layers.Dense(units=L1,activation='relu',name='FeedForward1'))
model.add(tf.contrib.keras.layers.Dense(units=L1,activation='relu',name='FeedForward2'))
#Output layer 
model.add(tf.contrib.keras.layers.Dense(units=Yfeatures,name='OutputLayer'))
#Specify los function and optimizer
model.compile(loss='mse',optimizer='adam',metrics=['mae'])

#Summarize model
model.summary()
#Callbacks
#tb_callback = tf.contrib.keras.callbacks.TensorBoard(log_dir='./logs',histogram_freq=2,write_images=True)
#,callbacks=[tb_callback]
# Start Training model
model.fit(X_train,y_train,batch_size=16,epochs=epochs,validation_split = 0.2,shuffle =True,verbose=1)
model.save('demo.h5')
print("Model Saved")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      (None, 3)                 0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3)                 12        
_________________________________________________________________
FeedForward1 (Dense)         (None, 10)                40        
_________________________________________________________________
FeedForward2 (Dense)         (None, 10)                110       
_________________________________________________________________
OutputLayer (Dense)          (None, 1)                 11        
Total params: 173
Trainable params: 167
Non-trainable params: 6
_________________________________________________________________
Train on 15788 samples, validate on 3947 samples
Epoch 1/10
15788/15788 [==============================] - 3s - loss: 14586.5739 - mean_absolute

In [68]:
#Inference session
model.predict(X_train,batch_size=10)

array([[ 139.65245056],
       [ 138.56143188],
       [ 137.50308228],
       ..., 
       [ 161.17932129],
       [ 160.55427551],
       [ 160.42619324]], dtype=float32)